In [1]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.decomposition import PCA

In [3]:
df = pd.read_csv('/content/99Acres_gurgaon_hyderabad_Secunderabad_SelectedFeatures_with_fewChange.csv')
pd.set_option('display.max_columns', None)

In [4]:
df.set_index('index', inplace=True)

In [5]:
df.head(2)

,PROPERTY_TYPE,OWNTYPE,BEDROOM_NUM,FURNISH,FACING,AGE,TOTAL_FLOOR,PRICE_SQFT,AREA,PRICE,BALCONY_NUM,FLOOR_NUM,BHK,LOCALITY,LATITUDE,LONGITUDE,LUXURY,SWIMMING_POOL,POWER_BACKUP,CLUB_HOUSE,ATM,GYM,WASTE_DISPOSAL,GAS_PIPELINE
index,,,,,,,,,,,,,,,,,,,,,,,,
1,Residential Apartment,1,3.0,4,3,5,30,5428.0,1658.0,9000000.0,3,9,3,Manikonda,17.580482,78.936868,Luxury,1,0,1,1,1,0,0
5,Independent/Builder Floor,1,2.0,4,2,3,2,7964.0,565.0,4500000.0,1,1,2,Manikonda,17.531600,78.267290,Luxury,0,0,0,0,0,0,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13224 entries, 1 to 18728
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PROPERTY_TYPE   13224 non-null  object 
 1   OWNTYPE         13224 non-null  int64  
 2   BEDROOM_NUM     13224 non-null  float64
 3   FURNISH         13224 non-null  int64  
 4   FACING          13224 non-null  int64  
 5   AGE             13224 non-null  int64  
 6   TOTAL_FLOOR     13224 non-null  int64  
 7   PRICE_SQFT      13224 non-null  float64
 8   AREA            13224 non-null  float64
 9   PRICE           13224 non-null  float64
 10  BALCONY_NUM     13224 non-null  int64  
 11  FLOOR_NUM       13224 non-null  int64  
 12  BHK             13224 non-null  int64  
 13  LOCALITY        13224 non-null  object 
 14  LATITUDE        13224 non-null  float64
 15  LONGITUDE       13224 non-null  float64
 16  LUXURY          13224 non-null  object 
 17  SWIMMING_POOL   13224 non-null  int6

In [ ]:
X = df.drop(columns=['PRICE','LOCALITY'])
y = df['PRICE']/10000000

In [ ]:
len(df['LOCALITY'].unique())

201

In [ ]:
# Applying the log1p transformation to the target variable
y_transformed = np.log1p(y)

In [ ]:
columns_to_encode = ['OWNTYPE', 'FACING','AGE', 'PROPERTY_TYPE', 'LUXURY', 'SWIMMING_POOL','CLUB_HOUSE','ATM','GYM','WASTE_DISPOSAL','GAS_PIPELINE','FURNISH','BALCONY_NUM','POWER_BACKUP']

In [ ]:
len(columns_to_encode)

14

In [ ]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['BEDROOM_NUM', 'TOTAL_FLOOR', 'PRICE_SQFT', 'AREA','LATITUDE','LONGITUDE','BHK','FLOOR_NUM']),
        ('cat', OrdinalEncoder(), columns_to_encode)
    ],
    remainder='passthrough'
)

In [ ]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [ ]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [ ]:
scores.mean(),scores.std()

(0.5702300989840363, 0.03749625715512571)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [ ]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['BEDROOM_NUM', 'TOTAL_FLOOR',
                                                   'PRICE_SQFT', 'AREA',
                                                   'LATITUDE', 'LONGITUDE',
                                                   'BHK', 'FLOOR_NUM']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['OWNTYPE', 'FACING', 'AGE',
                                                   'PROPERTY_TYPE', 'LUXURY',
                                                   'SWIMMING_POOL',
                                                   'CLUB_HOUSE', 'ATM', 'GYM',
                                                   'WASTE_DISPOSAL',
                                                   'GAS_PIPELINE', 'FURNISH',
                                                   'BALCONY_NUM',
                                                   'POWER_BACKUP'])])),
                ('regressor', LinearRegression())])

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
y_pred = np.expm1(y_pred)

In [ ]:
mean_absolute_error(np.expm1(y_test),y_pred)

0.927488310626302

In [ ]:
def scorer(model_name, model):

    output = []

    output.append(model_name)

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

    output.append(scores.mean())

    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

    pipeline.fit(X_train,y_train)

    y_pred = pipeline.predict(X_test)

    y_pred = np.expm1(y_pred)

    output.append(mean_absolute_error(np.expm1(y_test),y_pred))

    return output

In [ ]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [ ]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [ ]:
model_output

[['linear_reg', 0.5702300989840363, 0.927488310626302],
 ['svr', 0.7991790934430043, 0.43230653002705943],
 ['ridge', 0.5702336168141012, 0.9274555581548503],
 ['LASSO', -0.0005675093893070749, 1.3412020883221176],
 ['decision tree', 0.7239582561682518, 0.34767934468809075],
 ['random forest', 0.8534029507844105, 0.2895703498434461],
 ['extra trees', 0.8502588845896855, 0.30131824472400126],
 ['gradient boosting', 0.8375973972702578, 0.37112497908835723],
 ['adaboost', 0.6420113466572979, 0.9363348982765076],
 ['mlp', 0.8224062378445721, 0.4410553441795948],
 ['xgboost', 0.8434790074205989, 0.3329235351413238]]

In [ ]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [ ]:
model_df.sort_values(['mae'])

,name,r2,mae
5,random forest,0.853403,0.289570
6,extra trees,0.850259,0.301318
10,xgboost,0.843479,0.332924
4,decision tree,0.723958,0.347679
7,gradient boosting,0.837597,0.371125
1,svr,0.799179,0.432307
9,mlp,0.822406,0.441055
2,ridge,0.570234,0.927456
0,linear_reg,0.570230,0.927488
8,adaboost,0.642011,0.936335


# **OneHotEncoding**

In [ ]:
X = df.drop(columns=['PRICE','LOCALITY'])
y = df['PRICE']/10000000

In [ ]:
y_transformed = np.log1p(y)

In [ ]:
columns_to_encode = ['OWNTYPE','AGE', 'LUXURY', 'SWIMMING_POOL','CLUB_HOUSE','ATM','GYM','WASTE_DISPOSAL','GAS_PIPELINE','FURNISH','BALCONY_NUM','POWER_BACKUP']

In [ ]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['BEDROOM_NUM', 'TOTAL_FLOOR', 'PRICE_SQFT', 'AREA','LATITUDE','LONGITUDE','BHK','FLOOR_NUM']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first'),['PROPERTY_TYPE', 'FACING'])
    ],
    remainder='passthrough'
)

In [ ]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [ ]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [ ]:
scores.mean()

0.5713816497059138

In [ ]:
scores.std()

0.03718743898928851

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [ ]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['BEDROOM_NUM', 'TOTAL_FLOOR',
                                                   'PRICE_SQFT', 'AREA',
                                                   'LATITUDE', 'LONGITUDE',
                                                   'BHK', 'FLOOR_NUM']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['OWNTYPE', 'AGE', 'LUXURY',
                                                   'SWIMMING_POOL',
                                                   'CLUB_HOUSE', 'ATM', 'GYM',
                                                   'WASTE_DISPOSAL',
                                                   'GAS_PIPELINE', 'FURNISH',
                                                   'BALCONY_NUM',
                                                   'POWER_BACKUP']),
                                                 ('cat1',
                                                  OneHotEncoder(drop='first'),
                                                  ['PROPERTY_TYPE',
                                                   'FACING'])])),
                ('regressor', LinearRegression())])

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
y_pred = np.expm1(y_pred)

In [ ]:
mean_absolute_error(np.expm1(y_test),y_pred)

0.9231149764997559

In [ ]:
def scorer(model_name, model):

    output = []

    output.append(model_name)

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

    output.append(scores.mean())

    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

    pipeline.fit(X_train,y_train)

    y_pred = pipeline.predict(X_test)

    y_pred = np.expm1(y_pred)

    output.append(mean_absolute_error(np.expm1(y_test),y_pred))

    return output

In [ ]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [ ]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [ ]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [ ]:
model_df.sort_values(['mae'])

,name,r2,mae
5,random forest,0.854713,0.288714
6,extra trees,0.846423,0.300971
10,xgboost,0.844103,0.337412
4,decision tree,0.720198,0.347891
7,gradient boosting,0.836714,0.366276
1,svr,0.806442,0.405332
9,mlp,0.817910,0.442163
2,ridge,0.571387,0.923076
0,linear_reg,0.571382,0.923115
8,adaboost,0.639224,0.939532


# **OneHotEncoding With PCA**

In [ ]:
columns_to_encode = ['OWNTYPE','AGE', 'LUXURY', 'SWIMMING_POOL','CLUB_HOUSE','ATM','GYM','WASTE_DISPOSAL','GAS_PIPELINE','FURNISH','BALCONY_NUM','POWER_BACKUP']

In [ ]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['BEDROOM_NUM', 'TOTAL_FLOOR', 'PRICE_SQFT', 'AREA','LATITUDE','LONGITUDE','BHK','FLOOR_NUM']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first'),['PROPERTY_TYPE', 'FACING'])
    ],
    remainder='passthrough'
)

In [ ]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('pca', PCA(n_components=0.95)),
    ('regressor', LinearRegression())
])

In [ ]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [ ]:
scores.mean()

0.5653290662996284

In [ ]:
scores.std()

0.038903056698863285

In [ ]:
def scorer(model_name, model):

    output = []

    output.append(model_name)

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('pca', PCA(n_components=0.90)),
        ('regressor', model)
    ])

    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

    output.append(scores.mean())

    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

    pipeline.fit(X_train,y_train)

    y_pred = pipeline.predict(X_test)

    y_pred = np.expm1(y_pred)

    output.append(mean_absolute_error(np.expm1(y_test),y_pred))

    return output

In [ ]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [ ]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [ ]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [ ]:
model_df.sort_values(['mae'])

,name,r2,mae
1,svr,0.798902,0.413474
9,mlp,0.819331,0.415491
10,xgboost,0.788833,0.499115
6,extra trees,0.788699,0.501642
5,random forest,0.772620,0.534932
7,gradient boosting,0.714582,0.641055
4,decision tree,0.522166,0.746688
2,ridge,0.393256,1.031753
0,linear_reg,0.393256,1.031759
8,adaboost,0.324678,1.134292


In [13]:
import category_encoders as ce

In [ ]:
columns_to_encode = ['OWNTYPE','AGE', 'LUXURY', 'SWIMMING_POOL','CLUB_HOUSE','ATM','GYM','WASTE_DISPOSAL','GAS_PIPELINE','FURNISH','BALCONY_NUM','POWER_BACKUP']

In [ ]:
X = df.drop(columns=['PRICE'])
y = df['PRICE']/10000000

In [ ]:
y_transformed = np.log1p(y)

In [ ]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['BEDROOM_NUM', 'TOTAL_FLOOR', 'PRICE_SQFT', 'AREA','LATITUDE','LONGITUDE','BHK','FLOOR_NUM']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first'),['PROPERTY_TYPE', 'FACING']),
        ('target_enc', ce.TargetEncoder(), ['LOCALITY'])
    ],
    remainder='passthrough'
)

In [ ]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [ ]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [ ]:
scores.mean(),scores.std()

(0.6188464752359134, 0.0343826438163756)

In [ ]:
def scorer(model_name, model):

    output = []

    output.append(model_name)

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

    output.append(scores.mean())

    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

    pipeline.fit(X_train,y_train)

    y_pred = pipeline.predict(X_test)

    y_pred = np.expm1(y_pred)

    output.append(mean_absolute_error(np.expm1(y_test),y_pred))

    return output


In [ ]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [ ]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [ ]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [ ]:
model_df.sort_values(['mae'])

,name,r2,mae
5,random forest,0.854951,0.284988
6,extra trees,0.846212,0.308518
10,xgboost,0.842512,0.330595
4,decision tree,0.727824,0.354864
7,gradient boosting,0.838557,0.373234
1,svr,0.810145,0.397955
9,mlp,0.819566,0.423628
0,linear_reg,0.618846,0.793003
2,ridge,0.618858,0.793084
8,adaboost,0.643557,0.936128


# **Hyperparameter Tuning**

In [7]:
from sklearn.model_selection import GridSearchCV

In [8]:
param_grid = {
    'regressor__n_estimators': [50, 100, 200, 300],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__max_samples':[0.1, 0.25, 0.5, 1.0],
    'regressor__max_features': ['auto', 'sqrt']
}

In [14]:
X = df.drop(columns=['PRICE','PRICE_SQFT','LATITUDE','LONGITUDE'])
y = df['PRICE']/10000000

In [15]:
# Applying the log1p transformation to the target variable
y_transformed = np.log1p(y)

In [16]:
columns_to_encode = ['OWNTYPE','AGE', 'LUXURY', 'SWIMMING_POOL','CLUB_HOUSE','ATM','GYM','WASTE_DISPOSAL','GAS_PIPELINE','FURNISH','BALCONY_NUM','POWER_BACKUP']

In [18]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['BEDROOM_NUM', 'TOTAL_FLOOR', 'AREA','BHK','FLOOR_NUM']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first'),['PROPERTY_TYPE', 'FACING']),
        ('target_enc', ce.TargetEncoder(), ['LOCALITY'])
    ],
    remainder='passthrough'
)

In [19]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

In [20]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

In [21]:
search = GridSearchCV(pipeline, param_grid, cv=kfold, scoring='r2', n_jobs=-1, verbose=4)

In [22]:
search.fit(X, y_transformed)

Fitting 10 folds for each of 128 candidates, totalling 1280 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


GridSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['BEDROOM_NUM',
                                                                          'TOTAL_FLOOR',
                                                                          'AREA',
                                                                          'BHK',
                                                                          'FLOOR_NUM']),
                                                                        ('cat',
                                                                         OrdinalEncoder(),
                                                                         ['OWNTYPE',
                                                                          'AGE',
                                                                          'LUXURY',
                                                                          'SWIMMING_POOL',
                                                                          'CLUB_HOUSE',
                                                                          'ATM',
                                                                          'GYM',
                                                                          'WASTE_DISPOSAL',...
                                                                         OneHotEncoder(drop='first'),
                                                                         ['PROPERTY_TYPE',
                                                                          'FACING']),
                                                                        ('target_enc',
                                                                         TargetEncoder(),
                                                                         ['LOCALITY'])])),
                                       ('regressor', RandomForestRegressor())]),
             n_jobs=-1,
             param_grid={'regressor__max_depth': [None, 10, 20, 30],
                         'regressor__max_features': ['auto', 'sqrt'],
                         'regressor__max_samples': [0.1, 0.25, 0.5, 1.0],
                         'regressor__n_estimators': [50, 100, 200, 300]},
             scoring='r2', verbose=4)

In [23]:
final_pipe = search.best_estimator_

In [24]:
search.best_params_

{'regressor__max_depth': 20,
 'regressor__max_features': 'auto',
 'regressor__max_samples': 0.5,
 'regressor__n_estimators': 300}

In [25]:
search.best_score_

0.8158682451073984

In [26]:
final_pipe.fit(X,y_transformed)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['BEDROOM_NUM', 'TOTAL_FLOOR',
                                                   'AREA', 'BHK',
                                                   'FLOOR_NUM']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['OWNTYPE', 'AGE', 'LUXURY',
                                                   'SWIMMING_POOL',
                                                   'CLUB_HOUSE', 'ATM', 'GYM',
                                                   'WASTE_DISPOSAL',
                                                   'GAS_PIPELINE', 'FURNISH',
                                                   'BALCONY_NUM',
                                                   'POWER_BACKUP']),
                                                 ('cat1',
                                                  OneHotEncoder(drop='first'),
                                                  ['PROPERTY_TYPE', 'FACING']),
                                                 ('target_enc', TargetEncoder(),
                                                  ['LOCALITY'])])),
                ('regressor',
                 RandomForestRegressor(max_depth=20, max_features='auto',
                                       max_samples=0.5, n_estimators=300))])

In [30]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['BEDROOM_NUM', 'TOTAL_FLOOR', 'AREA','BHK','FLOOR_NUM']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first'),['PROPERTY_TYPE', 'FACING']),
        ('target_enc', ce.TargetEncoder(), ['LOCALITY'])
    ],
    remainder='passthrough'
)

In [31]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=500))
])

In [32]:
pipeline.fit(X,y_transformed)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['BEDROOM_NUM', 'TOTAL_FLOOR',
                                                   'AREA', 'BHK',
                                                   'FLOOR_NUM']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['OWNTYPE', 'AGE', 'LUXURY',
                                                   'SWIMMING_POOL',
                                                   'CLUB_HOUSE', 'ATM', 'GYM',
                                                   'WASTE_DISPOSAL',
                                                   'GAS_PIPELINE', 'FURNISH',
                                                   'BALCONY_NUM',
                                                   'POWER_BACKUP']),
                                                 ('cat1',
                                                  OneHotEncoder(drop='first'),
                                                  ['PROPERTY_TYPE', 'FACING']),
                                                 ('target_enc', TargetEncoder(),
                                                  ['LOCALITY'])])),
                ('regressor', RandomForestRegressor(n_estimators=500))])

In [33]:
import pickle

with open('model.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

In [34]:
with open('df.pkl', 'wb') as file:
    pickle.dump(X, file)

In [35]:
X.columns

Index(['PROPERTY_TYPE', 'OWNTYPE', 'BEDROOM_NUM', 'FURNISH', 'FACING', 'AGE',
       'TOTAL_FLOOR', 'AREA', 'BALCONY_NUM', 'FLOOR_NUM', 'BHK', 'LOCALITY',
       'LUXURY', 'SWIMMING_POOL', 'POWER_BACKUP', 'CLUB_HOUSE', 'ATM', 'GYM',
       'WASTE_DISPOSAL', 'GAS_PIPELINE'],
      dtype='object')

In [36]:
X.iloc[0].values

array(['Residential Apartment', 1, 3.0, 4, 3, 5, 30, 1658.0, 3, 9, 3,
       'Manikonda', 'Luxury', 1, 0, 1, 1, 1, 0, 0], dtype=object)

In [38]:
df['LOCALITY'].unique()

array(['Manikonda', 'Sector 84', 'Sector 81', 'Sector 66', 'Sector 112',
       'Sector 104', 'Sector 74', 'Sector 102', 'Sector 60', 'Sector 110',
       'Sector 109', 'Sector 65', 'Sector 37D', 'Sector 113',
       'Sector 111', 'Sector 61', 'Block B Sushant Lok Phase - 3',
       'Sector 62', 'Sector 57', 'Sector 59', 'Sector 37C', 'Sector 53',
       'Sector 28', 'Sector 67', 'Sector 54', 'Sector 106', 'Sector 108',
       'DLF Phase 2', 'Sector 82', 'South City 2', 'DLF Phase 5',
       'Sector 55', 'Sector 48', 'Sector 90', 'Sector 103',
       'Block C, Sushant Lok Phase 1', 'Sector 63', 'DLF Phase 1',
       'Sector 107', 'Malibu Town', 'Sector 63A', 'Uppals Southend',
       'Sector 85', 'Sector 68', 'Sushant Lok Phase 1', 'Sector-33 Sohna',
       'Nirvana Country', 'Rosewood', 'Sector 50', 'Sector 36A',
       'Sector 110 A', 'Sector 51', 'Sector 31', 'Palam Vihar',
       'Sector 23', 'Sector 46', 'Sector 45', 'South City 1',
       'Sector 67A', 'Sohna', 'Sector 36 Sohna',

In [39]:
data = [['Manikonda',1, 5, 'Luxury', 1, 1, 1, 1, 1, 1, 2, 2, 1,3,4,2750,3,2,'Residential Apartment',1]]
columns = ['LOCALITY','OWNTYPE','AGE', 'LUXURY', 'SWIMMING_POOL','CLUB_HOUSE','ATM','GYM','WASTE_DISPOSAL','GAS_PIPELINE','FURNISH','BALCONY_NUM','POWER_BACKUP','BEDROOM_NUM', 'TOTAL_FLOOR', 'AREA','BHK','FLOOR_NUM','PROPERTY_TYPE', 'FACING']

# Convert to DataFrame
one_df = pd.DataFrame(data, columns=columns)

one_df

,LOCALITY,OWNTYPE,AGE,LUXURY,SWIMMING_POOL,CLUB_HOUSE,ATM,GYM,WASTE_DISPOSAL,GAS_PIPELINE,FURNISH,BALCONY_NUM,POWER_BACKUP,BEDROOM_NUM,TOTAL_FLOOR,AREA,BHK,FLOOR_NUM,PROPERTY_TYPE,FACING
0,Manikonda,1,5,Luxury,1,1,1,1,1,1,2,2,1,3,4,2750,3,2,Residential Apartment,1


In [40]:
np.expm1(pipeline.predict(one_df))

array([2.00051844])

In [41]:
X.dtypes

PROPERTY_TYPE      object
OWNTYPE             int64
BEDROOM_NUM       float64
FURNISH             int64
FACING              int64
AGE                 int64
TOTAL_FLOOR         int64
AREA              float64
BALCONY_NUM         int64
FLOOR_NUM           int64
BHK                 int64
LOCALITY           object
LUXURY             object
SWIMMING_POOL       int64
POWER_BACKUP        int64
CLUB_HOUSE          int64
ATM                 int64
GYM                 int64
WASTE_DISPOSAL      int64
GAS_PIPELINE        int64
dtype: object